In [1]:
import matplotlib.pyplot as plt
from transformers import AutoImageProcessor, AutoModel
import torch

import cv2
import numpy as np

from sklearn.model_selection import train_test_split
import os
import tqdm

c:\gr\TestTask\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Prepare data

In [2]:
targets = []
paths = []

for folder in os.listdir("./flowers"):
    targets.extend([folder] * len(os.listdir(os.path.join("./flowers", folder))))
    paths.extend(
        [
            os.path.join("./flowers", folder, file)
            for file in os.listdir(os.path.join("./flowers", folder))
        ]
    )

In [3]:
train_paths, test_paths, train_targets, test_targtes = train_test_split(paths, targets, test_size=0.2, stratify=targets, shuffle=True, random_state=42)

# Feature extraction

## Pretrained 

In [4]:
preprocess = AutoImageProcessor.from_pretrained("facebook/dinov2-small", device="cuda:0")
model = AutoModel.from_pretrained("facebook/dinov2-small")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
with torch.no_grad():
    embeddings_train = []
    for path in tqdm.tqdm(train_paths):
        embeddings_train.append(
            model(
                **preprocess(
                    cv2.imread(path),
                    return_tensors="pt",
                )
            ).pooler_output
        )
        
    embeddings_test = []
    for path in tqdm.tqdm(test_paths):
        embeddings_test.append(
            model(
                **preprocess(
                    cv2.imread(path),
                    return_tensors="pt",
                )
            ).pooler_output
        )


 82%|████████▏ | 2823/3453 [04:15<00:59, 10.66it/s]

## Self made